In [4]:
#from lib2to3.pgen2.token import BACKQUOTE
import os
from pickletools import optimize
from tabnanny import verbose
from turtle import shape
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import math
from tensorflow.keras import layers
from tensorflow import keras

#physical_devices = tf.config.list_physical_devices("GPU")
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files = False,
    as_supervised = True,
    with_info=True,
)

@tf.function
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

@tf.function
def rotate(img, max_degrees=25):
    degrees = tf.random.uniform([], -max_degrees, max_degrees, dtype = tf.float32)
    img = tfa.image.rotate(img, degrees*math.pi/ 180, interpolation="BILINEAR")
    return image


@tf.function
def augment(image,label):
    image = tf.image.resize(image, size=[28,28])

    #coloring of image
    image = rotate(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_constrast(image, lower=0.5, upper=1.5)

    #do augmentation here
    return image, label


AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

def my_model():
    inputs = keras.Input(shape=(28,28,1))
    x = layers.Conv2D(32, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(10, activation = "softmax")(x)
    return keras.Model(inputs=inputs, outputs= outputs)

model = my_model()
model.compile(
    loss= keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer = keras.optimizers.Adam(lr=1e-4),
    metrics = ["accuracy"],
)

model.fit(ds_train, epochs=30, verbose=2)
model.evaluate(ds_test)
model.save("model")

# compile model
# model.fit
# model.evaluate
# after training convert to tflite model

NameError: in user code:

    File "C:\Users\abhis\AppData\Local\Temp\ipykernel_10288\1679346813.py", line 41, in augment  *
        image = rotate(image)
    File "C:\Users\abhis\AppData\Local\Temp\ipykernel_10288\1679346813.py", line 33, in rotate  *
        return image

    NameError: name 'image' is not defined


In [7]:
## import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import math
from tensorflow.keras import layers
from tensorflow import keras

#physical_devices = tf.config.list_physical_devices("GPU")
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=False,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset
)

@tf.function
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

@tf.function
def color(x):
    x = tf.image.random_brightness(x, max_delta=0.20)
    x = tf.image.random_contrast(x, lower=0.4, upper=1.6)
    return x


@tf.function
def rotate(x, max_degrees=50):
    degrees = tf.random.uniform([], -max_degrees, max_degrees, dtype=tf.int64)
    x = tfa.image.rotate(x, tf.cast(degrees, tf.float32) * math.pi / 180, interpolation='BILINEAR')
    return x

@tf.function
def augment(image, label):
    image = tf.image.resize(image, size=[28, 28])
    image = rotate(image, max_degrees=25)
    image = color(image)
    return image, label


AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

# Setup for train dataset
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

# Setup for test Dataset
ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

def my_model():
    inputs = keras.Input(shape=(28, 28, 1))
    x = layers.Conv2D(32, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3)(x)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, activation="relu")(x)
    outputs = layers.Dense(10, activation="softmax")(x)
    return keras.Model(inputs=inputs, outputs=outputs)

model = my_model()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer=keras.optimizers.Adam(lr=1e-4),
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=20, verbose=2)
model.evaluate(ds_test)
model.save("model")

Epoch 1/20
1875/1875 - 66s - loss: 0.2885 - accuracy: 0.9209 - 66s/epoch - 35ms/step
Epoch 2/20
1875/1875 - 67s - loss: 0.0888 - accuracy: 0.9738 - 67s/epoch - 36ms/step
Epoch 3/20
1875/1875 - 62s - loss: 0.0648 - accuracy: 0.9803 - 62s/epoch - 33ms/step
Epoch 4/20
1875/1875 - 59s - loss: 0.0562 - accuracy: 0.9834 - 59s/epoch - 31ms/step
Epoch 5/20
1875/1875 - 57s - loss: 0.0467 - accuracy: 0.9863 - 57s/epoch - 30ms/step
Epoch 6/20
1875/1875 - 57s - loss: 0.0422 - accuracy: 0.9869 - 57s/epoch - 31ms/step
Epoch 7/20
1875/1875 - 59s - loss: 0.0388 - accuracy: 0.9883 - 59s/epoch - 32ms/step
Epoch 8/20
1875/1875 - 55s - loss: 0.0347 - accuracy: 0.9897 - 55s/epoch - 29ms/step
Epoch 9/20
1875/1875 - 59s - loss: 0.0337 - accuracy: 0.9900 - 59s/epoch - 31ms/step
Epoch 10/20
1875/1875 - 59s - loss: 0.0276 - accuracy: 0.9915 - 59s/epoch - 32ms/step
Epoch 11/20
1875/1875 - 58s - loss: 0.0272 - accuracy: 0.9916 - 58s/epoch - 31ms/step
Epoch 12/20
1875/1875 - 57s - loss: 0.0259 - accuracy: 0.9919 -

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [8]:
import tensorflow as tf

saved_dir = "model"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
model_lite = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(model_lite)